## Final Project Submission

Please fill out:
* Student name: Laura Lewis
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: Joe San Pietro
* Blog post URL:
***

# Introduction

Project aims, data source, etc.

***
# Obtaining the data

In this section...

Source of data:...

In [1]:
# Importing the required libraries
import pandas as pd
pd.set_option('display.max_columns', 50) # Display up to 50 columns at a time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob # To read all csv files in the directory

The most recent Kickstarter data from https://webrobots.io/kickstarter-datasets/ (from 14 March 2019) is stored in 56 separate csv files. The code below creates a list of all csv files beginning with 'Kickstarter' and concatenates them into one dataframe:

In [317]:
df = pd.concat([pd.read_csv(f) for f in glob.glob('data/Kickstarter*.csv')], ignore_index = True)

In [84]:
df.head(3)

,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,current_currency,deadline,disable_communication,friends,fx_rate,goal,id,is_backing,is_starrable,is_starred,launched_at,location,name,permissions,photo,pledged,profile,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,21,2006 was almost 7 years ago.... Can you believ...,"{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""po...",802,US,1387659690,"{""id"":1495925645,""name"":""Daniel"",""is_registere...",USD,$,True,USD,1391899046,False,NaN,1.0,200.0,287514992,NaN,False,NaN,1388011046,"{""id"":2379574,""name"":""Chicago"",""slug"":""chicago...",New Final Round Album,NaN,"{""key"":""assets/011/625/534/5bea1760d7f20943c4c...",802.0,"{""id"":822687,""project_id"":822687,""state"":""inac...",new-final-round-album,https://www.kickstarter.com/discover/categorie...,True,False,successful,1391899046,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",802.0,international
1,97,An adorable fantasy enamel pin series of princ...,"{""id"":54,""name"":""Mixed Media"",""slug"":""art/mixe...",2259,US,1549659768,"{""id"":1175589980,""name"":""Katherine"",""slug"":""fr...",USD,$,True,USD,1551801611,False,NaN,1.0,400.0,385129759,NaN,False,NaN,1550073611,"{""id"":2486340,""name"":""Sacramento"",""slug"":""sacr...",Princess Pals Enamel Pin Series,NaN,"{""key"":""assets/024/033/030/dea4e3901d10195b035...",2259.0,"{""id"":3665299,""project_id"":3665299,""state"":""in...",princess-pals-enamel-pin-series,https://www.kickstarter.com/discover/categorie...,True,False,successful,1551801611,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",2259.0,international
2,88,Helping a community come together to set the s...,"{""id"":280,""name"":""Photobooks"",""slug"":""photogra...",29638,US,1477242384,"{""id"":1196856269,""name"":""MelissaThomas"",""is_re...",USD,$,True,USD,1480607930,False,NaN,1.0,27224.0,681033598,NaN,False,NaN,1478012330,"{""id"":2383660,""name"":""Columbus"",""slug"":""columb...",Their Life Through Their Lens-the Amish and Me...,NaN,"{""key"":""assets/014/262/672/97944960ba30239051d...",29638.0,"{""id"":2730508,""project_id"":2730508,""state"":""in...",their-life-through-their-lens-the-amish-and-me...,https://www.kickstarter.com/discover/categorie...,True,True,successful,1480607932,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",29638.0,international


The resulting dataframe contains 209,222 projects.

In [85]:
len(df)

209222

***
# Scrubbing the data

In this section...

**Inspecting and dropping columns**

In [86]:
# Checking the columns
df.columns

Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'created_at', 'creator', 'currency', 'currency_symbol',
       'currency_trailing_code', 'current_currency', 'deadline',
       'disable_communication', 'friends', 'fx_rate', 'goal', 'id',
       'is_backing', 'is_starrable', 'is_starred', 'launched_at', 'location',
       'name', 'permissions', 'photo', 'pledged', 'profile', 'slug',
       'source_url', 'spotlight', 'staff_pick', 'state', 'state_changed_at',
       'static_usd_rate', 'urls', 'usd_pledged', 'usd_type'],
      dtype='object')

Description of each column...

- 
- disable_communication - whether or not a project owner disabled communication with their backers
- is_starrable - whether or not a project has been starred (liked and saved) by users

In [320]:
# Checking for duplicates of individual projects
print(f"Of the {len(df)} projects in the dataset, there are {len(df[df.duplicated(subset='id')])} which are listed more than once.")

Of the 209222 projects in the dataset, there are 26958 which are listed more than once.


Duplicates are an issue in this dataset and will need to be dealt with. Further cleaning of the data will help clarify which duplicates, if any, need to be removed.

In [88]:
# Checking column information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209222 entries, 0 to 209221
Data columns (total 37 columns):
backers_count               209222 non-null int64
blurb                       209214 non-null object
category                    209222 non-null object
converted_pledged_amount    209222 non-null int64
country                     209222 non-null object
created_at                  209222 non-null int64
creator                     209222 non-null object
currency                    209222 non-null object
currency_symbol             209222 non-null object
currency_trailing_code      209222 non-null bool
current_currency            209222 non-null object
deadline                    209222 non-null int64
disable_communication       209222 non-null bool
friends                     300 non-null object
fx_rate                     209222 non-null float64
goal                        209222 non-null float64
id                          209222 non-null int64
is_backing                  300 

Some columns contain very few nun-null entries, and can be dropped:

In [321]:
# Dropping columns that are mostly null
df.drop(['friends', 'is_backing', 'is_starred', 'permissions'], axis=1, inplace=True)

Other columns are not useful for the purposes of this project, and can also be dropped:

- converted_pledged_amount - most currencies are converted into USD in this column, but not all. Instead, the 'usd_pledged' column will be used as these all use the same currency (the dollar).
- creator - most projects are by different people, and so this cannot be usefully used to group or categorise projects, and is not useful in a machine learning context.
- currency - all currency values will be used as/converted to dollars, so that they can be evaluated together. It is not necessary to keep the original record because of this, and because it will be highly correlated with country (which will be kept).
- currency_symbol - as above.
- currency_trailing_code - as above.
- current_currency - as above.
- fx_rate - this is used to create 'converted_pledged_amount' from 'pledged', but does not always convert to dollars so can be dropped in favour of 'static_usd_rate' which always converts to dollars.
- photo - image processing/computer vision will not be used in this project.
- pledged - data in this column is stored in native currencies, so this will be dropped in favour of 'usd_pledged' which is all in the same currency (dollars).
- profile - this column contains a combination of information from other columns (e.g. id, state, dates, url).
- slug - this is simply the 'name' column with hyphens instead of spaces.
- source_url - the sites that the rows were each scraped from is not useful for building a model, as each is unique to an id.
- spotlight - projects can only be spotlighted after they are already successful, so this will be entirely correlated with successful projects.
- state_changed_at - this is the same as deadline for most projects. The only exceptions are for projects which were cancelled before their deadline, but they will not be included in this analysis.
- urls - as with source_url.
- usd_type - it is unclear what this column means, but it is unlikely to be necessary since all currency values will be converted to dollars, and other currency information has been dropped.

In [322]:
# Dropping columns that aren't useful
df.drop(['converted_pledged_amount', 'creator', 'currency', 'currency_symbol', 'currency_trailing_code', 'current_currency', 'fx_rate', 'photo', 'pledged', 'profile', 'slug', 'source_url', 'spotlight', 'state_changed_at', 'urls', 'usd_type'], axis=1, inplace=True)

**Converting datetime columns**

Columns containing dates are currently stored in unix time, and need to be converted to datetime. Because they have been converted from unix, all times are in UTC/GMT.

In [323]:
# Converting dates from unix to datetime
cols_to_convert = ['created_at', 'deadline', 'launched_at']
for c in cols_to_convert:
    df[c] = pd.to_datetime(df[c], origin='unix', unit='s')

In [93]:
print(f"The dataset contains projects added to Kickstarter between {min(df.created_at).strftime('%d %B %Y')} and {max(df.created_at).strftime('%d %B %Y')}.")

The dataset contains projects added to Kickstarter between 22 April 2009 and 14 March 2019.


Individual columns will now be pre-processed, and additional features engineered, where necessary.

**Blurb**

Natural language processing is beyond the scope of this project. The length of the blurbs written by project creators will be calculated though, in case this is useful for the model (e.g. people preferring to read shorter or longer blurbs when choosing what to fund). The original blurb variable will then be dropped.

In [324]:
# Count length of each blurb
df['blurb_length'] = df['blurb'].str.split().str.len()

# Drop blurb variable
df.drop('blurb', axis=1, inplace=True)

**Category**

The category variable is currently stored as a string, although it was clearly originally a dictionary. The example below shows that each project has both a category (e.g. games) and a sub-category (e.g. tabletop games). Both will be extracted.

In [95]:
# Example category value
df.iloc[0]['category']

'{"id":43,"name":"Rock","slug":"music/rock","position":17,"parent_id":14,"color":10878931,"urls":{"web":{"discover":"http://www.kickstarter.com/discover/categories/music/rock"}}}'

In [325]:
# Extracting the relevant sub-category section from the string
f = lambda x: x['category'].split('/')[1].split('","position')[0]
df['sub_category'] = df.apply(f, axis=1)

# Extracting the relevant category section from the string, and replacing the original category variable
f = lambda x: x['category'].split('"slug":"')[1].split('/')[0]
df['category'] = df.apply(f, axis=1)

In [97]:
# Counting the number of unique categories
print(f"There are {df.category.nunique()} unique categories and {df.sub_category.nunique()} unique sub-categories.")

There are 30 unique categories and 145 unique sub-categories.


**Disable_communication**

99.7% of project owners did not disable communication with their backers (unsurprisingly). Because nearly all projects have the same value for this variable, it will be dropped as it does not provide much information.

In [98]:
# Checking the proportions of each category
df.disable_communication.value_counts(normalize=True)

False    0.997022
True     0.002978
Name: disable_communication, dtype: float64

In [326]:
df.drop('disable_communication', axis=1, inplace=True)

**Goal**

The goal amount of funding for each project is currently recorded in native currencies. In order to allow for fair comparisons between projects, goal will be converted into dollars (as amount pledged already is).

In [336]:
# Calculate new column 'usd_goal' as goal * static_usd_rate
df['usd_goal'] = round(df['goal'] * df['static_usd_rate'],2)

In [337]:
# Dropping goal and static_usd_rate
df.drop(['goal', 'static_usd_rate'], axis=1, inplace=True)

**Is_starrable**

Only 3.3% of projects were starred by users. Although this is only a very small proportion, whether or not a project was liked and saved by users is likely to be informative about whether or not a project was successful, so the variable will be kept.

In [102]:
# Figure out what this is, and do a count_values() to figure out whether it's worth including or mostly FALSE
df.is_starrable.value_counts(normalize=True)

False    0.967403
True     0.032597
Name: is_starrable, dtype: float64

**Location**

The location field contains the town/city that a project originates from, as well as the country. There are a large number (15,235) of unique locations. Because the country is already recorded separately in the country field, and there are such a large number of unique categories (making one-hot encoding not useful, particularly as there are likely to be a lot of smaller towns and cities with very few projects), the column will be dropped.

In [103]:
# Example location value
df.iloc[0]['location']

'{"id":2379574,"name":"Chicago","slug":"chicago-il","short_name":"Chicago, IL","displayable_name":"Chicago, IL","localized_name":"Chicago","country":"US","state":"IL","type":"Town","is_root":false,"urls":{"web":{"discover":"https://www.kickstarter.com/discover/places/chicago-il","location":"https://www.kickstarter.com/locations/chicago-il"},"api":{"nearby_projects":"https://api.kickstarter.com/v1/discover?signature=1552595044.c1041c6bca69b0b72738f3b9504ebf921b3e5e0e&woe_id=2379574"}}}'

In [104]:
# Counting the number of unique locations
df.location.nunique()

15235

In [339]:
# Dropping location
df.drop('location', axis=1, inplace=True)

**Name**

The length of project names will be calculated, in case this is useful for the model. The original name variable will then be dropped.

In [340]:
# Count length of each name
df['name_length'] = df['name'].str.split().str.len()
# Drop name variable
df.drop('name', axis=1, inplace=True)

**Usd_pledged**

This column requires rounding to two decimal places.

In [341]:
df['usd_pledged'] = round(df['usd_pledged'],2)

**Additional calculated features**

Additional features can be calculated from the existing features, which may also help to predict whether a project is successfully funded. The features to be added are: time from creation to launch, campaign length, launch day of week, deadline day of week, launch month, deadline month, launch time of day, deadline time of day. Original datetime values will be kept in for now for EDA purposes, but will be removed later, before modeling.

In [343]:
# Time between creating and launching a project
df['creation_to_launch_days'] = df['launched_at'] - df['created_at']
df['creation_to_launch_days'] = df['creation_to_launch_days'].dt.round('d').dt.days # Rounding to nearest days, then showing as number only
# Or could show as number of hours:
# df['creation_to_launch_hours'] = df['launched_at'] - df['created_at']
# df['creation_to_launch_hours'] = df['creation_to_launch_hours'].dt.round('h') / np.timedelta64(1, 'h') 

# Campaign length
df['campaign_days'] = df['deadline'] - df['launched_at']
df['campaign_days'] = df['campaign_days'].dt.round('d').dt.days # Rounding to nearest days, then showing as number only

# Launch day of week
df['launch_day'] = df['launched_at'].dt.weekday_name

# Deadline day of week
df['deadline_day'] = df['deadline'].dt.weekday_name

# Launch month
df['launch_month'] = df['launched_at'].dt.month_name()

# Deadline month
df['deadline_month'] = df['deadline'].dt.month_name()

# Launch time
df['launch_time'] = df['launched_at'].dt.round('h').dt.hour

# Deadline time
df['deadline_time'] = df['deadline'].dt.round('h').dt.hour

# TO BE FIXED - BIN LAUNCH AND DEADLINE TIME INTO TWO HOUR CATEGORIES

In [247]:
#df['launch_time'] = pd.cut(df.launched_at.dt.time, (pd.date_range('00:00:00', '23:59:59', freq='2H').time))

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'

In [219]:
#df['launch_time'] = df['launched_at'].dt.round('h').dt.hour # Round to nearest hour and extract hours from datetime
#df["launch_time"] = pd.cut(df["launch_time"], bins=[0,2,4,6,8,12,14,16,18,20,22,24], right=False) # Bin
#print(len(df.launch_time.unique()))
#df.head()

11


,backers_count,category,country,created_at,deadline,id,is_starrable,launched_at,staff_pick,state,usd_pledged,blurb_length,sub_category,usd_goal,name_length,campaign_days,creation_to_launch_days,launch_day,deadline_day,launch_month,deadline_month,launch_time
0,21,music,US,2013-12-21 21:01:30,2014-02-08 22:37:26,287514992,False,2013-12-25 22:37:26,False,successful,802.00000,26.0,rock,200.0000,4,45,4,Wednesday,Saturday,December,February,"[22, 24)"
1,97,art,US,2019-02-08 21:02:48,2019-03-05 16:00:11,385129759,False,2019-02-13 16:00:11,False,successful,2259.00000,9.0,mixed media,400.0000,5,20,5,Wednesday,Tuesday,February,March,"[16, 18)"
2,88,photography,US,2016-10-23 17:06:24,2016-12-01 15:58:50,681033598,False,2016-11-01 14:58:50,True,successful,29638.00000,25.0,photobooks,27224.0000,9,30,9,Tuesday,Thursday,November,December,"[14, 16)"
3,193,fashion,IT,2018-10-24 08:32:00,2018-12-08 22:59:00,1031782682,False,2018-10-27 23:56:22,False,successful,49075.15252,13.0,footwear,45461.0028,5,42,4,Saturday,Saturday,October,December,"[0, 2)"
4,20,technology,US,2015-03-07 05:35:17,2015-04-08 16:36:57,904085819,False,2015-03-09 16:36:57,False,failed,549.00000,22.0,software,1000.0000,4,30,2,Monday,Wednesday,March,April,"[16, 18)"


In [257]:
#df.head()

,backers_count,category,country,created_at,deadline,id,is_starrable,launched_at,staff_pick,state,usd_pledged,blurb_length,sub_category,usd_goal,name_length,campaign_days,creation_to_launch_days,launch_day,deadline_day,launch_month,deadline_month,launch_time,deadline_time
0,21,music,US,2013-12-21 21:01:30,2014-02-08 22:37:26,287514992,False,2013-12-25 22:37:26,False,successful,802.00000,26.0,rock,200.0000,4,45,4,Wednesday,Saturday,December,February,23,23
1,97,art,US,2019-02-08 21:02:48,2019-03-05 16:00:11,385129759,False,2019-02-13 16:00:11,False,successful,2259.00000,9.0,mixed media,400.0000,5,20,5,Wednesday,Tuesday,February,March,16,16
2,88,photography,US,2016-10-23 17:06:24,2016-12-01 15:58:50,681033598,False,2016-11-01 14:58:50,True,successful,29638.00000,25.0,photobooks,27224.0000,9,30,9,Tuesday,Thursday,November,December,15,16
3,193,fashion,IT,2018-10-24 08:32:00,2018-12-08 22:59:00,1031782682,False,2018-10-27 23:56:22,False,successful,49075.15252,13.0,footwear,45461.0028,5,42,4,Saturday,Saturday,October,December,0,23
4,20,technology,US,2015-03-07 05:35:17,2015-04-08 16:36:57,904085819,False,2015-03-09 16:36:57,False,failed,549.00000,22.0,software,1000.0000,4,30,2,Monday,Wednesday,March,April,17,17


**Checking for null values**

In [270]:
df.isna().sum()

backers_count              0
category                   0
country                    0
created_at                 0
deadline                   0
id                         0
is_starrable               0
launched_at                0
staff_pick                 0
state                      0
usd_pledged                0
blurb_length               8
sub_category               0
usd_goal                   0
name_length                0
campaign_days              0
creation_to_launch_days    0
launch_day                 0
deadline_day               0
launch_month               0
deadline_month             0
launch_time                0
deadline_time              0
dtype: int64

There are eight projects without a blurb_length, i.e. without a blurb. These can be replaced with a length of 0.

In [344]:
# Replacing null values for blurb_length with 0
df.blurb_length.fillna(0, inplace=True)

In [345]:
# Confirming there are no null values remaining
df.isna().sum().sum()

0

**Dropping rows**

This project aims to predict whether projects succeed or fail. The dataset also includes canceled, live (i.e. not yet finished) and suspended projects. These will now be removed.

In [278]:
# Number of projects of different states
df.state.value_counts()

successful    117465
failed         75199
canceled        8624
live            7311
suspended        623
Name: state, dtype: int64

In [346]:
# Dropping projects which are not successes or failures
df = df[df['state'].isin(['successful', 'failed'])]

In [286]:
# Confirming that the most recent deadline is the day on which the data was scraped, i.e. there are no projects which have yet to be resolved into either successes or failures
max(df.deadline)

Timestamp('2019-03-14 04:06:01')

**Dropping duplicates**

As demonstrated above, some projects are included in the dataset more than once. Duplicates will now be assessed and removed.

In [347]:
# Checking for duplicates of individual projects, and sorting by id
duplicates = df[df.duplicated(subset='id')]
print(f"Of the {len(df)} projects in the dataset, there are {len(df[df.duplicated(subset='id')])} which are listed more than once.")
print(f"Of these, {len(df[df.duplicated()])} have every value in common between duplicates.")

Of the 192664 projects in the dataset, there are 23685 which are listed more than once.
Of these, 23674 have every value in common between duplicates.


In [348]:
# Dropping duplicates which have every value in common
df.drop_duplicates(inplace=True)

In [349]:
len(df)

168990

In [365]:
print(len(df[df.duplicated(subset='id')]), "duplicated projects remain.")
duplicated = df[df.duplicated(subset='id', keep=False)].sort_values(by='id')
duplicated

11 duplicated projects remain.


,backers_count,category,country,created_at,deadline,id,is_starrable,launched_at,staff_pick,state,usd_pledged,blurb_length,sub_category,usd_goal,name_length,creation_to_launch_days,campaign_days,launch_day,deadline_day,launch_month,deadline_month,launch_time,deadline_time
197835,1675,games,NZ,2018-05-30 04:51:55,2018-12-18 04:00:00,122079667,False,2018-11-21 02:04:29,False,successful,200537.60,12.0,tabletop games,342.10,4,175,27,Wednesday,Tuesday,November,December,2,4
144698,1675,games,NZ,2018-05-30 04:51:55,2018-12-18 04:00:00,122079667,False,2018-11-21 02:04:29,False,successful,200710.36,12.0,tabletop games,342.40,4,175,27,Wednesday,Tuesday,November,December,2,4
124657,32,art,CA,2018-05-07 22:52:48,2018-05-30 01:00:00,123735637,False,2018-05-08 01:43:43,False,successful,669.56,13.0,illustration,466.97,10,0,22,Tuesday,Wednesday,May,May,2,1
106307,32,art,CA,2018-05-07 22:52:48,2018-05-30 01:00:00,123735637,False,2018-05-08 01:43:43,False,successful,669.10,13.0,illustration,466.65,10,0,22,Tuesday,Wednesday,May,May,2,1
105017,18,photography,FR,2017-11-06 23:50:45,2018-01-06 02:53:04,299750696,False,2017-12-02 02:53:04,False,successful,1654.77,18.0,photobooks,1423.46,8,25,35,Saturday,Saturday,December,January,3,3
191946,18,photography,FR,2017-11-06 23:50:45,2018-01-06 02:53:04,299750696,False,2017-12-02 02:53:04,False,successful,1661.61,18.0,photobooks,1429.34,8,25,35,Saturday,Saturday,December,January,3,3
39735,779,games,CA,2018-12-23 02:00:22,2019-03-02 03:00:00,395324593,False,2019-01-29 00:02:43,False,successful,10222.68,17.0,tabletop games,151.33,7,37,32,Tuesday,Saturday,January,March,0,3
102505,779,games,CA,2018-12-23 02:00:22,2019-03-02 03:00:00,395324593,False,2019-01-29 00:02:43,False,successful,10231.08,17.0,tabletop games,151.46,7,37,32,Tuesday,Saturday,January,March,0,3
31755,5,technology,HK,2018-03-15 11:39:29,2018-04-27 02:09:50,1188888346,False,2018-03-28 02:09:50,False,successful,1437.70,20.0,gadgets,1274.10,11,13,30,Wednesday,Friday,March,April,2,2
84748,5,technology,HK,2018-03-15 11:39:29,2018-04-27 02:09:50,1188888346,False,2018-03-28 02:09:50,False,successful,1437.92,20.0,gadgets,1274.30,11,13,30,Wednesday,Friday,March,April,2,2


Comparing rows for each duplicated project:

In [406]:
# Get list of index numbers for duplicated ids
dup_ids = duplicated.id.unique()
for i in dup_ids:
    index1 = duplicated[duplicated.id == i][:1].index.values
    index2 = duplicated[duplicated.id == i][1:2].index.values
    print(index1, index2)
    #print(duplicated.loc[index1] == duplicated.loc[index2]) # produces TypeError: Could not compare [None] with block values

[197835] [144698]
[124657] [106307]
[105017] [191946]
[39735] [102505]
[31755] [84748]
[204234] [69815]
[165028] [2380]
[195186] [198134]
[103127] [63824]
[165952] [58250]
[94567] [184176]


In [407]:
df.loc[197835] == df.loc[144698]

backers_count               True
category                    True
country                     True
created_at                  True
deadline                    True
id                          True
is_starrable                True
launched_at                 True
staff_pick                  True
state                       True
usd_pledged                False
blurb_length                True
sub_category                True
usd_goal                   False
name_length                 True
creation_to_launch_days     True
campaign_days               True
launch_day                  True
deadline_day                True
launch_month                True
deadline_month              True
launch_time                 True
deadline_time               True
dtype: bool

In [417]:
df.loc[124657] == df.loc[106307]

backers_count               True
category                    True
country                     True
created_at                  True
deadline                    True
id                          True
is_starrable                True
launched_at                 True
staff_pick                  True
state                       True
usd_pledged                False
blurb_length                True
sub_category                True
usd_goal                   False
name_length                 True
creation_to_launch_days     True
campaign_days               True
launch_day                  True
deadline_day                True
launch_month                True
deadline_month              True
launch_time                 True
deadline_time               True
dtype: bool

In [416]:
df.loc[105017] == df.loc[191946]

backers_count               True
category                    True
country                     True
created_at                  True
deadline                    True
id                          True
is_starrable                True
launched_at                 True
staff_pick                  True
state                       True
usd_pledged                False
blurb_length                True
sub_category                True
usd_goal                   False
name_length                 True
creation_to_launch_days     True
campaign_days               True
launch_day                  True
deadline_day                True
launch_month                True
deadline_month              True
launch_time                 True
deadline_time               True
dtype: bool

In [415]:
df.loc[39735] == df.loc[102505]

backers_count               True
category                    True
country                     True
created_at                  True
deadline                    True
id                          True
is_starrable                True
launched_at                 True
staff_pick                  True
state                       True
usd_pledged                False
blurb_length                True
sub_category                True
usd_goal                   False
name_length                 True
creation_to_launch_days     True
campaign_days               True
launch_day                  True
deadline_day                True
launch_month                True
deadline_month              True
launch_time                 True
deadline_time               True
dtype: bool

In [414]:
df.loc[31755] == df.loc[84748]

backers_count               True
category                    True
country                     True
created_at                  True
deadline                    True
id                          True
is_starrable                True
launched_at                 True
staff_pick                  True
state                       True
usd_pledged                False
blurb_length                True
sub_category                True
usd_goal                   False
name_length                 True
creation_to_launch_days     True
campaign_days               True
launch_day                  True
deadline_day                True
launch_month                True
deadline_month              True
launch_time                 True
deadline_time               True
dtype: bool

In [413]:
df.loc[204234] == df.loc[69815]

backers_count               True
category                    True
country                     True
created_at                  True
deadline                    True
id                          True
is_starrable                True
launched_at                 True
staff_pick                  True
state                       True
usd_pledged                False
blurb_length                True
sub_category                True
usd_goal                   False
name_length                 True
creation_to_launch_days     True
campaign_days               True
launch_day                  True
deadline_day                True
launch_month                True
deadline_month              True
launch_time                 True
deadline_time               True
dtype: bool

In [412]:
df.loc[165028] == df.loc[2380]

backers_count               True
category                    True
country                     True
created_at                  True
deadline                    True
id                          True
is_starrable                True
launched_at                 True
staff_pick                  True
state                       True
usd_pledged                False
blurb_length                True
sub_category                True
usd_goal                   False
name_length                 True
creation_to_launch_days     True
campaign_days               True
launch_day                  True
deadline_day                True
launch_month                True
deadline_month              True
launch_time                 True
deadline_time               True
dtype: bool

In [411]:
df.loc[195186] == df.loc[198134]

backers_count               True
category                    True
country                     True
created_at                  True
deadline                    True
id                          True
is_starrable                True
launched_at                 True
staff_pick                  True
state                       True
usd_pledged                False
blurb_length                True
sub_category                True
usd_goal                   False
name_length                 True
creation_to_launch_days     True
campaign_days               True
launch_day                  True
deadline_day                True
launch_month                True
deadline_month              True
launch_time                 True
deadline_time               True
dtype: bool

In [410]:
df.loc[103127] == df.loc[63824]

backers_count               True
category                    True
country                     True
created_at                  True
deadline                    True
id                          True
is_starrable                True
launched_at                 True
staff_pick                  True
state                       True
usd_pledged                False
blurb_length                True
sub_category                True
usd_goal                   False
name_length                 True
creation_to_launch_days     True
campaign_days               True
launch_day                  True
deadline_day                True
launch_month                True
deadline_month              True
launch_time                 True
deadline_time               True
dtype: bool

In [409]:
df.loc[165952] == df.loc[58250]

backers_count               True
category                    True
country                     True
created_at                  True
deadline                    True
id                          True
is_starrable                True
launched_at                 True
staff_pick                  True
state                       True
usd_pledged                False
blurb_length                True
sub_category                True
usd_goal                   False
name_length                 True
creation_to_launch_days     True
campaign_days               True
launch_day                  True
deadline_day                True
launch_month                True
deadline_month              True
launch_time                 True
deadline_time               True
dtype: bool

In [408]:
df.loc[94567] == df.loc[184176]

backers_count               True
category                    True
country                     True
created_at                  True
deadline                    True
id                          True
is_starrable                True
launched_at                 True
staff_pick                  True
state                       True
usd_pledged                False
blurb_length                True
sub_category                True
usd_goal                   False
name_length                 True
creation_to_launch_days     True
campaign_days               True
launch_day                  True
deadline_day                True
launch_month                True
deadline_month              True
launch_time                 True
deadline_time               True
dtype: bool

The results above show that for each pair of duplicates, there are differences in the 

In [370]:
duplicated.id.unique()

array([ 122079667,  123735637,  299750696,  395324593, 1188888346,
       1212887301, 1287336933, 1323539307, 1573483573, 1702392394,
       1934536281], dtype=int64)

In [391]:
duplicated[duplicated.id == 122079667][:1].index.values

array([197835], dtype=int64)

In [398]:
duplicated[duplicated.id == 122079667][0:1].sort_index(inplace=True) == duplicated[duplicated.id == 122079667][1:2].sort_index(inplace=True)

True

In [366]:
df.loc[197835] == df.loc[144698]

backers_count               True
category                    True
country                     True
created_at                  True
deadline                    True
id                          True
is_starrable                True
launched_at                 True
staff_pick                  True
state                       True
usd_pledged                False
blurb_length                True
sub_category                True
usd_goal                   False
name_length                 True
creation_to_launch_days     True
campaign_days               True
launch_day                  True
deadline_day                True
launch_month                True
deadline_month              True
launch_time                 True
deadline_time               True
dtype: bool

In [292]:
df[df.duplicated(keep=False)].sort_values(by='id')

In [308]:
len(df[df.duplicated(subset='id')])

273

In [298]:
len(df[df.duplicated()])

23412

In [294]:
print(len(all_duplicates))
all_duplicates.head()

46824


,backers_count,category,country,created_at,deadline,id,is_starrable,launched_at,staff_pick,state,usd_pledged,blurb_length,sub_category,usd_goal,name_length,campaign_days,creation_to_launch_days,launch_day,deadline_day,launch_month,deadline_month,launch_time,deadline_time
100253,68,food,US,2016-05-15 18:58:21,2016-06-30 22:34:18,39036,False,2016-05-16 22:34:18,False,successful,3239.000000,13.0,small batch,2500.00000,4,45,1,Monday,Thursday,May,June,23,23
187623,68,food,US,2016-05-15 18:58:21,2016-06-30 22:34:18,39036,False,2016-05-16 22:34:18,False,successful,3239.000000,13.0,small batch,2500.00000,4,45,1,Monday,Thursday,May,June,23,23
33515,338,games,IE,2018-07-05 10:46:58,2018-10-10 06:48:50,39235,False,2018-09-10 06:48:50,False,successful,15440.032893,14.0,tabletop games,5781.91765,7,30,67,Monday,Wednesday,September,October,7,7
165153,338,games,IE,2018-07-05 10:46:58,2018-10-10 06:48:50,39235,False,2018-09-10 06:48:50,False,successful,15440.032893,14.0,tabletop games,5781.91765,7,30,67,Monday,Wednesday,September,October,7,7
108972,52,design,US,2018-04-05 19:35:16,2018-06-01 18:22:31,50419,False,2018-05-02 18:22:31,False,successful,2985.000000,10.0,graphic design,1250.00000,6,30,27,Wednesday,Friday,May,June,18,18


**Setting the index**

In [149]:
# Setting the id column as the index
df.set_index('id', inplace=True)
df.head()

,backers_count,category,country,created_at,deadline,disable_communication,goal,is_starrable,launched_at,location,staff_pick,state,state_changed_at,static_usd_rate,usd_pledged,blurb_length,name_length,sub_category
id,,,,,,,,,,,,,,,,,,
287514992,21,rock,US,2013-12-21 21:01:30,2014-02-08 22:37:26,False,200.0,False,2013-12-25 22:37:26,"{""id"":2379574,""name"":""Chicago"",""slug"":""chicago...",False,successful,2014-02-08 22:37:26,1.000000,802.00000,26.0,4,rock
385129759,97,mixed media,US,2019-02-08 21:02:48,2019-03-05 16:00:11,False,400.0,False,2019-02-13 16:00:11,"{""id"":2486340,""name"":""Sacramento"",""slug"":""sacr...",False,successful,2019-03-05 16:00:11,1.000000,2259.00000,9.0,5,mixed media
681033598,88,photobooks,US,2016-10-23 17:06:24,2016-12-01 15:58:50,False,27224.0,False,2016-11-01 14:58:50,"{""id"":2383660,""name"":""Columbus"",""slug"":""columb...",True,successful,2016-12-01 15:58:52,1.000000,29638.00000,25.0,9,photobooks
1031782682,193,footwear,IT,2018-10-24 08:32:00,2018-12-08 22:59:00,False,40000.0,False,2018-10-27 23:56:22,"{""id"":725746,""name"":""Venice"",""slug"":""venice-ve...",False,successful,2018-12-08 22:59:00,1.136525,49075.15252,13.0,5,footwear
904085819,20,software,US,2015-03-07 05:35:17,2015-04-08 16:36:57,False,1000.0,False,2015-03-09 16:36:57,"{""id"":2479651,""name"":""Redmond"",""slug"":""redmond...",False,failed,2015-04-08 16:36:59,1.000000,549.00000,22.0,4,software


***
# Exploring the data

In this setion...

In [ ]:
# Before modelling, save features minus dates as new dataframe

***
# Modeling

***
# Interpretation

***
# Summary